In [3]:
import requests
import bs4

In [9]:
with requests.Session() as s:

    p = s.post("https://www.chicagocityscape.com", data={
        "email": 'reclaimfairelections@gmail.com',
        "password": 'rDxPK7p6W@5f'})

    base_page = s.get('https://www.chicagocityscape.com/ordinances.php#/?search_term=O2019-331')
    soup = bs4.BeautifulSoup(base_page.content, 'html.parser')


In [23]:
for f in soup.find_all('table', id='table_ordinances'):
    print(f.next_element.next_element)

<div id="table_tools_ordinances">
<div class="row">
<div class="col-md-2">
<h4>Download the data</h4>
</div>
<div class="col-md-10">
<div class="insert_here"></div>
<p>These data download buttons require a <a href="/benefits.php?src=table_tools">Cityscape Pro membership</a></p>
</div>
</div><!-- /.row -->
</div>
